In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder
import tensorflow as tf

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

# def run_all_experiments(df, model_param_sets, output_dir):
#     total = len(model_param_sets)
#     print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
#     for index, params in enumerate(model_param_sets, 1):
#         print(f"Running Exp {index}/{total}...")
#         groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)


try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    # groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


C:\Users\steve\source\repos\grocery-ml
Building Training DF: Start
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Training DF: Done


In [2]:
import optuna 
def new_objective(trial):

    lr = trial.suggest_float("learning_rate", 0.001, 0.001)
    depth = trial.suggest_int("depth", 5, 20, step=5)
    base_units = trial.suggest_int("base_units", 150, 2048, step=12)
    decay = trial.suggest_float("decay", 0.2, 0.7)
    embedding_dim = trial.suggest_int("embedding_dim", 200, 300, step=10)
    epochs = trial.suggest_int("epochs", 30, 200, step=10)
    output_activation = "sigmoid"
    metrics = ["AUC", "Precision", "Recall"]
    
    layers_cfg = []
    current_units = base_units
    
    for i in range(depth):
        layers_cfg.append({
            "units": int(current_units),
            "activation": "relu"
        })
        current_units = max(4, current_units * decay)

    build_params = {
        "embedding_dim": embedding_dim,
        "layers": layers_cfg,
        "output_activation": output_activation,
        "optimizer": "adam",
        "learning_rate": lr,
        "loss": "binary_crossentropy" if output_activation == "sigmoid" else "mse",
        "metrics": metrics
    }

    train_params = { "epochs": epochs, "batch_size": 32 }
    days = 10
    start_date = pd.Timestamp.now()
    outputDir = f"f:/exp/keras/optuna/{trial.study.study_name}/trial-{trial.number}"
    groceryML.run_experiment_with_consecutive_predictions( groceryML.training_df, build_params, train_params,  outputDir, start_date,  days)                     
    # groceryML.run_experiment(groceryML.training_df, build_params, train_params, outputDir)

    return groceryML.last_val_auc
############################################################################


In [4]:
sampler = optuna.samplers.TPESampler()

study_name = f"stuff-fixed-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
study = optuna.create_study(
    study_name= study_name,
    sampler=sampler,
    direction="maximize",   # or minimize — see note below
    storage="sqlite:///optuna_grocery.db",
    load_if_exists=True
)


study.optimize(new_objective, n_trials=300)

[I 2026-01-19 01:48:29,542] A new study created in RDB with name: stuff-fixed-20260119_014829
C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\optuna\distributions.py:684: UserWarning: The distribution is specified by [150, 2048] and step=12, but the range is not divisible by `step`. It will be replaced with [150, 2046].
  warnings.warn(


Creating dir: f:/exp/keras/optuna/stuff-fixed-20260119_014829/trial-0\e300_l1902-1123-663-392-231_ep2
run_experiment_with_consecutive_predictions() exp_dir: f:/exp/keras/optuna/stuff-fixed-20260119_014829/trial-0\e300_l1902-1123-663-392-231_ep2
when: 2026-01-19 01:48:29.590477 params: {'epochs': 200, 'batch_size': 32}
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-19 01:48:29.589476
Building Live DF: Start
_build_combined_df()
_build_sources()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:349: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Live DF: Done
Writing XLSX: ./live_df.xlsx
   XLSX Done: ./live_df.xlsx
_build_latest_rows_df: start
_build_latest_rows_df: done
create_item_supply_level_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
normalize_features()
6/6 [==============================] - 0s 4ms/step
build_prediction_input() prediction_date=2026-01-20 01:48:29.589476
Building Live DF: Start
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative

C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:349: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


...

compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Live DF: Done
Writing XLSX: ./live_df.xlsx
   XLSX Done: ./live_df.xlsx
_build_latest_rows_df: start
_build_latest_rows_df: done
create_item_supply_level_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
normalize_features()
6/6 [==============================] - 0s 4ms/step
build_prediction_input() prediction_date=2026-01-26 20:16:43.124764
Building Live DF: Start
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:349: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
_build_trip_level_feats()
build_school_schedule_features(): start
build_school_schedule_features(): done
build_holiday_features()
build_trip_interveral_feautres(): start
build_trip_interveral_feautres(): done
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
Building Live DF: Done
Writing XLSX: ./live_df.xlsx


[W 2026-01-19 20:24:35,709] Trial 124 failed with parameters: {'learning_rate': 0.001, 'depth': 15, 'base_units': 1590, 'decay': 0.636483283084892, 'embedding_dim': 230, 'epochs': 90} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_11604\2400184797.py", line 37, in new_objective
    groceryML.run_experiment_with_consecutive_predictions( groceryML.training_df, build_params, train_params,  outputDir, start_date,  days)
  File "C:\Users\steve\source\repos\grocery-ml\grocery_ml_tensorflow.py", line 446, in run_experiment_with_consecutive_predictions
    artifacts = self.build_prediction_input(prediction_date, norm_params)
  File "C:\Users\steve\source\repos\grocery-ml\grocery_ml_tensorflow.py", line 518, in build_prediction_input
    ts = dat

KeyboardInterrupt: 